In [0]:
import numpy as np
import tensorflow as tf
import math
import logging
logging.basicConfig(level=logging.DEBUG)
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from pandas import DataFrame
 
import pandas as pd
 
x = pd.read_csv("predx_for_regression.csv", header=0)

x_array = np.asarray(x, dtype = "float")

x_arrayt=x_array

y = pd.read_csv("predy_for_regression.csv", header=0)
y_array = np.asarray(y, dtype = "float")
y_arrayt=y_array

whole_data=np.concatenate((x_arrayt, y_arrayt),axis=1)

angle = pd.read_csv("angle.csv", header=0)
angle_array = np.asarray(angle, dtype = "float")

angle_arrayt=angle_array.transpose()

#Network parameters
n_hidden1 = 147
n_hidden2 = 74
n_hidden3 = 50
n_hidden4 = 50
n_input = 98
n_output = 5
#Learning parameters
learning_constant = 0.03#0.002
number_epochs = 200#5000#10000
batch_size = 10000 

#Defining the input and the output
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])

#DEFINING WEIGHTS AND BIASES，initailly, they are just random values


#Biases first hidden layer
b1 = tf.Variable(tf.random_normal([n_hidden1]))
#Biases second hidden layer
b2 = tf.Variable(tf.random_normal([n_hidden2]))
#Biases output layer
b3 = tf.Variable(tf.random_normal([n_hidden3]))
########################################################
b4 = tf.Variable(tf.random_normal([n_hidden4]))
########################################################
#Biases output layer
b5 = tf.Variable(tf.random_normal([n_output]))


#Weights connecting input layer with first hidden layer
w1 = tf.Variable(tf.random_normal([n_input, n_hidden1]))
#Weights connecting first hidden layer with second hidden layer
w2 = tf.Variable(tf.random_normal([n_hidden1, n_hidden2]))
#Weights connecting second hidden layer with output layer
w3 = tf.Variable(tf.random_normal([n_hidden2, n_hidden3]))
###########################################################
w4 = tf.Variable(tf.random_normal([n_hidden3, n_hidden4]))
##########################################################
w5 = tf.Variable(tf.random_normal([n_hidden4, n_output]))



#The incoming data given to the
#network is input_d
def multilayer_perceptron(input_d):
    #drop out###########################################
    w1_temp= tf.nn.dropout(w1, 0.5)
    w2_temp= tf.nn.dropout(w2, 0.5)
    w3_temp= tf.nn.dropout(w3, 0.5)
    w4_temp= tf.nn.dropout(w4, 0.5)
    ############################################
    #Task of neurons of first hidden layer
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(input_d, w1_temp), b1))
    #Task of neurons of second hidden layer
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, w2_temp), b2))
    #Task of neurons of output layer
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, w3_temp), b3))
    #########################################################
    #Task of neurons of output layer
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, w4_temp), b4))
    #########################################################
    #Task of neurons of output layer
    out_layer = tf.add(tf.matmul(layer_4, w5),b5)
    
    return out_layer

#Create model
neural_network = multilayer_perceptron(X)

#Define loss and optimizer
loss_op = tf.sqrt(tf.reduce_mean(tf.math.squared_difference(neural_network,Y)))+ 0.01*tf.nn.l2_loss(w1) + 0.01*tf.nn.l2_loss(w2) + 0.01*tf.nn.l2_loss(w3) + 0.01*tf.nn.l2_loss(w4) 
# reduce_mean didn't set axis, to get the overall average value. the outcome is a number in this case

#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=neural_network,labels=Y))
#optimizer = tf.train.GradientDescentOptimizer(learning_constant).minimize(loss_op)
 optimizer = tf.train.AdamOptimizer(learning_constant).minimize(loss_op)

#Initializing the variables
init = tf.global_variables_initializer()

label=angle_array

batch_x=(whole_data-200)/2000 #(367616, 98)
# temp=np.array([angle_array[:,0]])##*only consider the first column of the angle    (1, 367616)
# batch_y=temp.transpose() #( 367616, 1)
batch_y=label

from sklearn.model_selection import train_test_split

batch_x_train,batch_x_test,batch_y_train,batch_y_test=train_test_split(batch_x,batch_y,test_size=0.3)

label_train,label_test=train_test_split(label,test_size=0.3)

with tf.Session() as sess:
    sess.run(init)

    #Training epoch
    for epoch in range(number_epochs):
        sess.run(optimizer, feed_dict={X: batch_x_train, Y: batch_y_train})
        
        #Display the epoch
        if epoch % 100 == 0 and epoch>10:
            print("Epoch:", '%d' % (epoch))
            ac = loss_op.eval({X: batch_x_train, Y: batch_y_train})
            print("Accuracy:",  ac)
    ##kfole########################
    print('doing kfold validation')
    Accuracys_list = []
    k = 10
    kf = KFold(n_splits=k)
    for train_index, test_index in kf.split(label):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = batch_x_train[train_index], batch_x_train[test_index]
        y_train, y_test = batch_y_train[train_index], batch_y_train[test_index]
        for epoch in range(number_epochs):
            sess.run(optimizer, feed_dict={X: X_train, Y: y_train})
        ac = loss_op.eval({X: X_test, Y: y_test})
        Accuracys_list.add(ac)
    
    print("Average Root Mean Square Error: ",Accuracys_list/k)
    print('kfold validation finished')
    ##############################
    
    # Test model
    pred = (neural_network)  # Apply softmax to logits
    accuracy=tf.keras.losses.MSE(pred,Y)#mse:short for "mean square error"
    print("MeanSquareError: Accuracy:", np.square(accuracy.eval({X: batch_x_train, Y: batch_y_train})).mean() )
    #tf.keras.evaluate(pred,batch_x)
    
    ###############################################
    
    accuracy=tf.keras.losses.MAE(pred,Y)
    print("MeanAbsoluteError: Accuracy:", np.square(accuracy.eval({X: batch_x_train, Y: batch_y_train})).mean() )
    accuracy=tf.keras.losses.MAPE(pred,Y)
    print("MeanAbsolutePercentageError: Accuracy:", np.square(accuracy.eval({X: batch_x_train, Y: batch_y_train})).mean() )
    accuracy=tf.keras.losses.MSLE(pred,Y)
    print("MeanSquaredLogarithmicError: Accuracy:", np.square(accuracy.eval({X: batch_x_train, Y: batch_y_train})).mean() )
    print("Prediction:", pred.eval({X: batch_x_train}))
    print(batch_y)

    #############################################
    
    output=neural_network.eval({X: batch_x_train})
    plt.plot(batch_y_train[:,0], 'r', output[:,0], 'b')
    plt.ylabel("X's prediction vs true value")
    plt.show()
    
    output=neural_network.eval({X: batch_x_train})
    plt.plot(batch_y_train[:,1], 'r', output[:,1], 'b')
    plt.ylabel("Y's prediction vs true value")
    plt.show()
    
    output=neural_network.eval({X: batch_x_train})
    plt.plot(batch_y_train[:,2], 'r', output[:,2], 'b')
    plt.ylabel("Z's prediction vs true value")
    plt.show()
    
    #############################################
    
   
    df = DataFrame(output)

    export_csv = df.to_csv ('output.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

    print (df)
    #Create a session